In [ ]:
from tmlf.python import tmlf_pybind, model_builder, workspace, optimizer
import pandas as pd
import numpy as np

In [ ]:
nrows=1000
df_train = pd.read_csv("../digit-recognizer/train.csv", nrows=nrows)
df_test = pd.read_csv("../digit-recognizer/test.csv", nrows=nrows)

In [ ]:
labels_np = df_train.label.to_numpy()
features_np = df_train.loc[:, df_train.columns != 'label'].to_numpy() / 255.0

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(features_np, labels_np, test_size=0.2, random_state=23)
X_test = df_test.to_numpy() / 255.0

In [ ]:
train_model_name = "train_model"
valid_model_name = "valid_model"
test_model_name = "test_model"

for model_name, X, y in zip(
    [train_model_name, valid_model_name, test_model_name],
    [X_train.astype(np.float32), X_valid.astype(np.float32), X_test.astype(np.float32)],
    [y_train.astype(np.float32), y_valid.astype(np.float32), None],
):
    if y is None:
        y = np.full(X.shape, -1, dtype=np.float32)
    workspace.feed_tensor(f"{model_name}_X", X)
    workspace.feed_tensor(f"{model_name}_y", y)
    workspace.feed_tensor(f"{model_name}_cursor", np.array([0], dtype=np.float32))

In [ ]:
def create_model(name, train=True, metric=True, batch_size=100, hidden_dim=150):
    model = model_builder.Model()
    
    # read input
    model.circular_batch(
        [f"{name}_X", f"{name}_y", f"{name}_cursor"],
        ["feat", "label", f"{name}_cursor"],
        batch_size=batch_size,
    )
    model.fc("feat", "fc1", in_dim=28 * 28, out_dim=hidden_dim)
    model.relu("fc1", "relu1")
    model.fc("relu1", "fc2", in_dim=hidden_dim, out_dim=hidden_dim)
    model.relu("fc2", "relu2")
    model.fc("relu2", "fc3", in_dim=hidden_dim, out_dim=10)
    model.softmax("fc3", "softmax")
    
    if train or metric:
        model.accuracy(["softmax", "label"], "accuracy")
        model.label_cross_entropy(["softmax", "label"], "xent")
        model.averaged_loss("xent", "loss")
        
    if train:
        model.add_backward_ops("loss")
        optimizer.build_sgd(model)
        
    return model

In [ ]:
train_model = create_model(train_model_name)
valid_model = create_model(valid_model_name, train=False, metric=True)
test_model = create_model(test_model_name, train=False, metric=False)

model_builder.run_net(train_model.init_net)

In [ ]:
num_batches = 1000
for itr in range(num_batches):
    # model_builder.run_net(train_model.net)
    pass